# Preparations

We first do some imports and then load the different evaluations of the neural solver and symbolic tools


In [ ]:
from neurosynt import NeuroSyntEvalDataset
from ml2.dtypes import CSVDict
from ml2.ltl.ltl_syn import LTLSynEvalDataset
from ml2.globals import LOCAL_STORAGE_DIR
from copy import copy

### Load SYNTCOMP 2022

Load the SYNTCOMP 2022 evaluation data from the starexec platform and convert it into a EvalDataset

Unfortunateley this takes quite some time (~4min)


In [ ]:
syntcomp_data: NeuroSyntEvalDataset = NeuroSyntEvalDataset.from_syntcomp(
    dtype=CSVDict, name="Syntcomp", filename=LOCAL_STORAGE_DIR + "/ltl-syn/syntcomp2022.zip"
)

### Load neurosynt evaluations

The following loads previous evaluations.

If one has performed own full evaluations, replace all `neurosynt-benchmarking-...` with the new evaluations (for example `neurosynt-bm-full-strix`) to fully reproduce all results.


NeuroSynt_Strix on CPU cluster


In [ ]:
benchmark_s: NeuroSyntEvalDataset = NeuroSyntEvalDataset.load("ltl-syn/neurosynt-benchmarking-6")  # type: ignore
benchmark_s.df["result_tool"] = benchmark_s.df.apply(
    lambda row: "NeuroSynt-S" if row["result_tool"] == "NeuroSynt" else row["result_tool"], axis=1
)

NeuroSynt_BoSy on CPU cluster


In [ ]:
benchmark_b: NeuroSyntEvalDataset = NeuroSyntEvalDataset.load("ltl-syn/neurosynt-benchmarking-7")  # type: ignore
benchmark_b.df["result_tool"] = benchmark_b.df.apply(
    lambda row: "NeuroSynt-B" if row["result_tool"] == "NeuroSynt" else row["result_tool"], axis=1
)

NeuroSynt_Strix on GPU cluster


In [ ]:
benchmark_g: NeuroSyntEvalDataset = NeuroSyntEvalDataset.load("ltl-syn/neurosynt-benchmarking-8")  # type: ignore
benchmark_g.df["result_tool"] = benchmark_g.df.apply(lambda row: row["result_tool"] + "-G", axis=1)

NeuroSynt_Strix on CPU cluster (Timeout 1h)


In [ ]:
benchmark_l: NeuroSyntEvalDataset = NeuroSyntEvalDataset.load("ltl-syn/neurosynt-benchmarking-9")  # type: ignore
benchmark_l.df["result_tool"] = benchmark_l.df.apply(
    lambda row: row["result_tool"] + "-L", axis=1
)  # Strix timeout 3600s

NeuroSynt_Strix on MPB


In [ ]:
benchmark_local: NeuroSyntEvalDataset = NeuroSyntEvalDataset.load("ltl-syn/neurosynt-benchmarking-10")  # type: ignore
benchmark_local.df["result_tool"] = benchmark_local.df.apply(
    lambda row: row["result_tool"] + "-Local", axis=1
)  # Strix timeout 3600s

Load second and third neural solver model (alternative model/ configuration)


In [ ]:
benchmark_120: NeuroSyntEvalDataset = NeuroSyntEvalDataset.load("ltl-syn/neurosynt-benchmarking-0")
benchmark_120.df["result_tool"] = benchmark_120.df.apply(
    lambda row: row["result_tool"] + "-120", axis=1
)

evaluation45 = NeuroSyntEvalDataset.from_LTLSynEvalDataset(
    LTLSynEvalDataset.load("ltl-syn/ht-45-eval-0/eval/0/1/csv_logger"), name="ht45eval"
)
evaluation45.df["result_tool"] = evaluation45.df.apply(
    lambda row: row["result_tool"] + "-45", axis=1
)

evaluation45 = NeuroSyntEvalDataset(
    df=evaluation45.smallest_samples(include_invalid=True),
    add_stats=False,
    dtype=CSVDict,
    name="ht45eval",
)

### Merge multiple evaluation into one evaluation.

for easier calculations of the virtual best solver


In [ ]:
merged = NeuroSyntEvalDataset.from_merge(
    [
        benchmark_s,
        benchmark_b,
        benchmark_g,
        benchmark_l,
        benchmark_local,
        syntcomp_data,
    ],
    name="merged",
    project="ltl-syn",
)

# Total Solves (Table 2)

in the following we describe, how the columns of Table 2 can be reproduced.

First, we create a virtual best solver for all configurations of each tool. This refelcts the _grouped configurations_ columns.
Then, the three columns follow.


In [ ]:
merged_neurosynt = NeuroSyntEvalDataset.from_merge(
    [
        benchmark_s,
        benchmark_120,
        evaluation45,
        benchmark_b,
        benchmark_g,
        benchmark_l,
    ],
    name="merged",
    project="ltl-syn",
)

collapsed_neurosynt = merged_neurosynt.collapse(
    collapse_dict={
        "NeuroSynt": list(
            filter(
                lambda x: x.startswith("NeuroSynt"),
                list(merged_neurosynt.df["result_tool"].value_counts().index),
            )
        ),
    },
    smallest=True,
    fastest=False,
)
collapsed_neurosynt.df = collapsed_neurosynt.df[
    collapsed_neurosynt.df["result_tool"] == "NeuroSynt"
]

merged_symbolic = NeuroSyntEvalDataset.from_merge(
    [
        benchmark_s,
        benchmark_b,
        benchmark_g,
        benchmark_l,
        syntcomp_data,
    ],
    name="merged",
    project="ltl-syn",
)

collapsed_symbolic = merged_symbolic.collapse(
    collapse_dict={
        "Strix": list(
            filter(
                lambda x: x.startswith("Strix"),
                list(merged_symbolic.df["result_tool"].value_counts().index),
            )
        ),
        "BoSy": list(
            filter(
                lambda x: x.startswith("BoSy"),
                list(merged_symbolic.df["result_tool"].value_counts().index),
            )
        ),
        "Otus": list(
            filter(
                lambda x: x.startswith("Otus"),
                list(merged_symbolic.df["result_tool"].value_counts().index),
            )
        ),
        "ltlsynt": list(
            filter(
                lambda x: x.startswith("ltlsynt"),
                list(merged_symbolic.df["result_tool"].value_counts().index),
            )
        ),
        "sdf": list(
            filter(
                lambda x: x.startswith("sdf"),
                list(merged_symbolic.df["result_tool"].value_counts().index),
            )
        ),
        "NeuroSynt": list(
            filter(
                lambda x: x.startswith("NeuroSynt"),
                list(merged_symbolic.df["result_tool"].value_counts().index),
            )
        ),
    },
    fastest=True,
)

collapsed_symbolic.df = collapsed_symbolic.df[collapsed_symbolic.df["result_tool"] != "NeuroSynt"]

collapsed = NeuroSyntEvalDataset.from_merge(
    [
        collapsed_neurosynt,
        collapsed_symbolic,
    ],
    name="collapsed",
    project="ltl-syn",
)

first column: total solved, best configuration


In [ ]:
best = {
    "Strix": (0, ""),
    "NeuroSynt": (0, ""),
    "sdf": (0, ""),
    "ltlsynt": (0, ""),
    "BoSy": (0, ""),
    "Otus": (0, ""),
}

for tool in list(merged.df["result_tool"].value_counts().index):
    solved = len(merged.solved_by([tool]))
    # print(tool)
    # print(solved)
    for el in best:
        if tool.startswith(el):
            if best[el][0] < solved:
                best[el] = solved, tool

print(best)

second column, grouped configurations


In [ ]:
for tool in list(collapsed.df["result_tool"].value_counts().index):
    solved = len(collapsed.solved_by([tool]))
    print(tool)
    print(solved)

third column: exclusively solved grouped configurations


In [ ]:
for el in best:
    print(el)
    print(
        len(
            collapsed.exclusively_solved(
                by_tool=el,
                out_of_tools=list(collapsed.df["result_tool"].value_counts().index),
            )
        )
    )

# Cactus Plot (Fig 8)

This is the cactus plot from Fig. 8

The first cell is how to create the plot, the second cell is the creation of the plot


In [ ]:
from typing import Dict, Tuple

import pandas as pd
import plotly.graph_objects as go


def sort_column(df: pd.DataFrame, column):
    df = df.reset_index()
    return df.sort_values(by=column).reset_index().drop("index", axis=1).reset_index()


def cactus_plot(dfs: Dict[str, Tuple[pd.DataFrame, Dict]], column="result_duration_par", log=True):
    def cactus(df: pd.DataFrame, name: str, line_format: Dict):
        df = sort_column(df, column)

        def column_sum(row):
            return df[: row["index"] + 1][column].sum()

        df["column_sum"] = df.apply(column_sum, axis=1)

        return go.Scatter(
            x=df["index"], y=df["column_sum"], name=name, mode="lines", line=line_format
        )

    fig = go.Figure()

    for k, (df, line_format) in dfs.items():
        fig.add_trace(cactus(df, k, line_format))
    if log:
        fig.update_yaxes(type="log", exponentformat="power", dtick="D3")

    fig.update_yaxes(
        title_text="total wall-clock time (s)",
        gridcolor="#e8e8e8",
    )
    fig.update_xaxes(
        title_text=" No. of solved benchmarks (Total: 1075)",
        tickmode="array",
        tickvals=[x * 25 for x in range(0, 44)],
        ticktext=[str(x * 25) if x % 2 == 0 else "" for x in range(0, 44)],
        range=[0, 1080],
        gridcolor="#e8e8e8",
    )
    fig.update_layout(
        font=dict(color="black"),
        showlegend=True,
        legend=dict(
            orientation="v",
            yanchor="bottom",
            y=0.05,
            xanchor="right",
            x=0.99,
            font=dict(
                size=13,
            ),
        ),
        height=400,
        width=900,
        margin=dict(l=0, r=10, t=10, b=0),
        template="plotly_white",
    )
    fig.show(config={"staticPlot": True})
    return fig

In [ ]:
fig = cactus_plot(
    {
        "Neural Solver": (
            merged.fastest_samples(out_of_tools=["NeuroSynt-G"]),
            dict(color="black"),
        ),
        "BoSy": (
            merged.fastest_samples(out_of_tools=["BoSy"]),
            dict(color="#009cc4", dash="dash"),
        ),
        "NeuroSynt<sub>BoSy": (
            merged.fastest_samples(out_of_tools=["NeuroSynt-G", "BoSy"]),
            dict(color="#009cc4"),
        ),
        "Strix": (
            merged.fastest_samples(out_of_tools=["Strix-L"]),
            dict(color="#ff7f23", dash="dash"),
        ),
        "NeuroSynt<sub>Strix": (
            merged.fastest_samples(out_of_tools=["NeuroSynt-G", "Strix-L"]),
            dict(color="#ff7f23"),
        ),
        "Virtual Best Solver<br><span style='font-size: 11px;'>(symbolic)</span>": (
            merged.fastest_samples(
                out_of_tools=list(
                    filter(
                        lambda x: not x.startswith("NeuroSynt"),
                        list(merged.df["result_tool"].value_counts().index),
                    )
                )
            ),
            dict(color="#008040", dash="dash"),
        ),
        "Virtual Best Solver<br><span style='font-size: 11px;'>(symbolic & neural)</span>": (
            merged.fastest_samples(),
            dict(color="#008040"),
        ),
    }
)

# Novel Solves


These are the names of the novel solves that we report in the paper.

These names correlate to the official files from SYNTCOMP 2022.
A copy can be found in `~/ml2-storage/ltl-spec/sc-1`


In [ ]:
collapsed.exclusively_solved(by_tool="NeuroSynt")["input_name"]

# Times (Table 3)

Here we reproduce the times from Table 3.

For each tool we list different hardware, mean and standard deviation.

For the neural solver, we additionally report the time for model checking.


### Neural solver


duration of neural solver (including model checking) on CPU cluster


In [ ]:
print(benchmark_b.solved_by(["NeuroSynt-B"])["result_duration_par"].mean())
print(benchmark_b.solved_by(["NeuroSynt-B"])["result_duration_par"].std())

duration of model checking for the neural solver on CPU cluster


In [ ]:
print(merged.solved_by(["NeuroSynt-B"])["result_model_checking_duration"].astype(float).mean())
print(merged.solved_by(["NeuroSynt-B"])["result_model_checking_duration"].astype(float).std())

duration of neural solver (including model checking) on MBP


In [ ]:
print(merged.solved_by(["NeuroSynt-Local"])["result_duration_par"].mean())
print(merged.solved_by(["NeuroSynt-Local"])["result_duration_par"].std())

duration of model checking for the neural solver on MPB


In [ ]:
print(merged.solved_by(["NeuroSynt-Local"])["result_model_checking_duration"].astype(float).mean())
print(merged.solved_by(["NeuroSynt-Local"])["result_model_checking_duration"].astype(float).std())

duration of neural solver (including model checking) on GPU cluster


In [ ]:
print(merged.solved_by(["NeuroSynt-G"])["result_duration_par"].mean())
print(merged.solved_by(["NeuroSynt-G"])["result_duration_par"].std())

duration of model checking for the neural solver on GPU cluster


In [ ]:
print(merged.solved_by(["NeuroSynt-G"])["result_model_checking_duration"].astype(float).mean())
print(merged.solved_by(["NeuroSynt-G"])["result_model_checking_duration"].astype(float).std())

### Symbolic Solver


Strix on GPU cluster


In [ ]:
print(merged.solved_by(["Strix-G"])["result_duration_par"].mean())
print(merged.solved_by(["Strix-G"])["result_duration_par"].std())

Strix on CPU cluster


In [ ]:
print(merged.solved_by(["Strix"])["result_duration_par"].mean())
print(merged.solved_by(["Strix"])["result_duration_par"].std())

Strix 1h timeout (CPU cluster)


In [ ]:
print(merged.solved_by(["Strix-L"])["result_duration_par"].mean())
print(merged.solved_by(["Strix-L"])["result_duration_par"].std())

Strix best SYNTCOMP config


In [ ]:
print(merged.solved_by(["Strix-ltl_synth_zlk_bfs"])["result_duration_par"].mean())
print(merged.solved_by(["Strix-ltl_synth_zlk_bfs"])["result_duration_par"].std())

Bosy on CPU cluster


In [ ]:
print(merged.solved_by(["BoSy"])["result_duration_par"].mean())
print(merged.solved_by(["BoSy"])["result_duration_par"].std())

# Sizes (Table 4, Fig 9)


## Table 4

here we describe how to reproduce the results from Table 4 line by line. We follow the same order as in the paper.


### Strix and BoSy


In [ ]:
r = merged.compare_smaller(by_tool="NeuroSynt-G", out_of_tools=["NeuroSynt-G", "Strix-L"])
print("On samples that Strix and the neural solver solved")
print("average Strix:", r["out_of_tools but not by_tools average"])
print("average  neural solver:", r["by_tool average"])
print(
    "neural solver smaller by",
    r["% smaller or larger than out_of_tools but not by_tools average"],
    "%",
)

In [ ]:
r = merged.compare_smaller(by_tool="NeuroSynt-G", out_of_tools=["NeuroSynt-G", "BoSy"])
print("On samples that BoSy and the neural solver solved")
print("average BoSy:", r["out_of_tools but not by_tools average"])
print("average  neural solver:", r["by_tool average"])
print(
    "neural solver smaller by",
    r["% smaller or larger than out_of_tools but not by_tools average"],
    "%",
)

### On realizable Syntcomp

We first merge our own evaluations of the neural solver with the SYNTCOMP 2022 results. For each tool, we create a virtual best solver that comprises all configurations of one tool. The virtual best criteria is circuit size. As the SYNTCOMP only reports systems for realizable specifications, we only consider realizable specifications in this context.


In [ ]:
merged_alt = NeuroSyntEvalDataset.from_merge(
    [
        benchmark_s,
        benchmark_120,
        evaluation45,
        benchmark_b,
        benchmark_g,
        benchmark_l,
        syntcomp_data,
    ],
    name="merged",
    project="ltl-syn",
)

collapsed_smallest = merged_alt.collapse(
    collapse_dict={
        "Strix": list(
            filter(
                lambda x: x.startswith("Strix"),
                list(merged_alt.df["result_tool"].value_counts().index),
            )
        ),
        "BoSy": list(
            filter(
                lambda x: x.startswith("BoSy"),
                list(merged_alt.df["result_tool"].value_counts().index),
            )
        ),
        "Otus": list(
            filter(
                lambda x: x.startswith("Otus"),
                list(merged_alt.df["result_tool"].value_counts().index),
            )
        ),
        "ltlsynt": list(
            filter(
                lambda x: x.startswith("ltlsynt"),
                list(merged_alt.df["result_tool"].value_counts().index),
            )
        ),
        "sdf": list(
            filter(
                lambda x: x.startswith("sdf"),
                list(merged_alt.df["result_tool"].value_counts().index),
            )
        ),
        "NeuroSynt": list(
            filter(
                lambda x: x.startswith("NeuroSynt"),
                list(merged_alt.df["result_tool"].value_counts().index),
            )
        ),
    },
    smallest=True,
    fastest=False,
    realizable_only=True,
)

Unfortunately we found a bug that led to wrong calculation of the intersections of samples (i.e. commonly solved) between the tools from SYNTCOMP and our neural solver.

This is in regard to the last three results of the Table 4 in the appendix. Following are the updated results.

The trend and what is being reported in the main part of the paper (_The neural solver produces smaller solutions than any other solver in SYNTCOMP 2002_) still holds.

We will correct that in the final version of the paper.


In [ ]:
r = collapsed_smallest.compare_smaller(by_tool="NeuroSynt", out_of_tools=["NeuroSynt", "ltlsynt"])
print("On samples that ltlsynt and the neural solver solved")
print("average ltlsynt:", r["out_of_tools but not by_tools average"])
print("average  neural solver:", r["by_tool average"])
print(
    "neural solver smaller by",
    r["% smaller or larger than out_of_tools but not by_tools average"],
    "%",
)

In [ ]:
collapsed_smallest.compare_smaller(by_tool="NeuroSynt", out_of_tools=["NeuroSynt", "Otus"])
print("On samples that Otus and the neural solver solved")
print("average Otus:", r["out_of_tools but not by_tools average"])
print("average  neural solver:", r["by_tool average"])
print(
    "neural solver smaller by",
    r["% smaller or larger than out_of_tools but not by_tools average"],
    "%",
)

In [ ]:
collapsed_smallest.compare_smaller(by_tool="NeuroSynt", out_of_tools=["NeuroSynt", "sdf"])
print("On samples that sdf and the neural solver solved")
print("average sdf:", r["out_of_tools but not by_tools average"])
print("average  neural solver:", r["by_tool average"])
print(
    "neural solver smaller by",
    r["% smaller or larger than out_of_tools but not by_tools average"],
    "%",
)

## Fig 9

Herre we plot the size of circuits (no. of latches) for Strix and the neural solver on commonly solved samples.

The first cell is how to create the plot, the second cell is the creation of the plot


In [ ]:
from typing import Tuple
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go


def group_by(df, group_fn, column):
    d = {
        k: group_fn(group)
        for k, group in [(k, df.loc[v]) for k, v in df.groupby(by=column).groups.items()]
    }
    return pd.DataFrame.from_dict(d, orient="index").dropna()


def min_max_scaling(series):
    return (series - series.min()) / (series.max() - series.min())


def plot_hist(
    df_1: Tuple[pd.DataFrame, str],
    df_2: Tuple[pd.DataFrame, str],
    column: str,
    group_fn,
):
    dataframe_1 = group_by(df_1[0], group_fn, column).reset_index()
    dataframe_2 = group_by(df_2[0], group_fn, column).reset_index()

    data = []
    data.append(
        go.Bar(
            name=df_1[1],
            x=dataframe_1["index"],
            y=dataframe_1[0],
            marker_color="#009cc4",
            # marker_color="black",
        )
    )
    data.append(
        go.Bar(
            name=df_2[1],
            x=dataframe_2["index"],
            y=dataframe_2[0],
            marker_color="#ff7f23",
            # marker_color="black",
        )
    )

    fig = go.Figure(data=data)
    fig.update_yaxes(
        title_text="no. of instances",
        dtick=25,
        gridcolor="#e8e8e8",
    )
    fig.update_xaxes(title_text=column, dtick=1, tick0=0)
    fig.update_layout(
        font=dict(color="black"),
        barmode="group",
        # showlegend=True,
        legend=dict(
            orientation="v",
            yanchor="top",
            y=0.95,
            xanchor="right",
            x=0.99,
            font=dict(size=13),
        ),
        colorscale={"sequential": px.colors.qualitative.G10},
        height=300,
        width=900,
        margin=dict(l=0, r=10, t=10, b=0),
        template="plotly_white",
    )
    fig.show()
    return fig

In [ ]:
df = copy(merged.solved_by(["NeuroSynt-G", "Strix-L"]))

df["latches"] = df.apply(NeuroSyntEvalDataset.get_latches, axis=1)

fig = plot_hist(
    (df[df["result_tool"] == "NeuroSynt-G"], "Neural Solver"),
    (df[df["result_tool"] == "Strix-L"], "Strix"),
    "latches",
    len,
)